In [8]:
import pandas as pd
df = pd.read_csv('bills.csv')
df

,text,summary,title,n_tokens,ada_v2,similarities
0,Agregamos valor a tu organización. Con nuestra...,Qué hace Ceocom? El objetivo principal de Ceocom,Qué hace Ceocom?,78,"[-0.016644887626171112, -0.0353408046066761, -...",0.768964
1,Partners. Ceocom posee una alianza estratégica...,Qué socios o partners tiene Ceocom? Partners d...,Qué socios o partners tiene Ceocom?,34,"[-0.004839136265218258, -0.0340556725859642, 0...",0.747821
2,Nuestro Equipo -César Gamarra (Gerente General...,Quiénes conforman o integran Ceocom? El equipo...,Quiénes conforman o integran Ceocom?,256,"[-0.009924538433551788, -0.01931372843682766, ...",0.767103
3,"En CEOCOM, nos enfocamos en brindar servicios ...",Cómo trabaja o procede Ceocom? Cuatro pilares ...,Cómo trabaja o procede Ceocom?,191,"[-0.019212400540709496, -0.0347551628947258, 0...",0.755978
4,Somos una organización líder en el campo de la...,Ramas de la empresa Ceocom,Cúales son las areas de trabajo de Ceocom?,139,"[-0.03700961917638779, -0.021702183410525322, ...",0.773525
5,Conectividad y Colaboración. Los servicios enf...,Conectividad y Colaboración Primer Pilar de Ce...,Conectividad y Colaboración,140,"[-0.02666584588587284, -0.01612384244799614, 0...",0.759354
6,Automatización y Generación Inteligente del va...,Automatización y Generación Inteligente del va...,Automatización y Generación Inteligente del valor,228,"[-0.035700805485248566, -0.03523502126336098, ...",0.756784
7,Inteligencia de Decisiones. Para disminuir la ...,Inteligencia de Decisiones Tercer Pilar de Ceocom,Inteligencia de Decisiones,219,"[-0.03782690316438675, -0.012233754619956017, ...",0.746362
8,Innovación y Mejora Continua. El ciclo no pued...,Innovación y Mejora Continua Cuarto Pilar de C...,Innovación y Mejora,267,"[-0.034642718732357025, -0.01117140892893076, ...",0.754225
9,Process Mining y Automatización. Revolucionand...,Process Mining y Automatización de Procesos Pr...,Process Mining y Automatización.,286,"[-0.03161485865712166, -0.01001456007361412, 0...",0.737449


In [9]:
import os
API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

print(API_KEY)
print(RESOURCE_ENDPOINT)

13368252c26745b58bb4d9429b8dc331
https://chatbot-ceocom-oai.openai.azure.com/


In [1]:
import openai
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

df = pd.read_csv('bills.csv')

df['ada_v2'] = df["text"].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002-version2-billsumdatasetsample'))

#IF 'to_print' is set to True, the function will also print the obtained ROWS, else it won't
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        #engine="text-embedding-ada-002" engine debe ser identico al deployment name del modelo
		engine="text-embedding-ada-002-version2-billsumdatasetsample"
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

user_query = input("Ingresar el texto: ")
res = search_docs(df, user_query, top_n=3) # Aqui va la pregunta a realizar y se muestran resultados con mas similitud
first_row = res.iloc[0]  # Selecciona la primera fila del df
text_value = first_row['text']  # Extraer el valor de la columna 'text'
print('Respuesta obtenida por parte del modelo:')
print(text_value)

,text,summary,title,n_tokens,ada_v2,similarities
4,Somos una organización líder en el campo de la...,Ramas de la empresa Ceocom,Cúales son las areas de trabajo de Ceocom?,139,"[-0.03700961917638779, -0.021702183410525322, ...",0.775893
2,Nuestro Equipo -César Gamarra (Gerente General...,Quiénes conforman o integran Ceocom? El equipo...,Quiénes conforman o integran Ceocom?,256,"[-0.009924538433551788, -0.01931372843682766, ...",0.774024
0,Agregamos valor a tu organización. Con nuestra...,Qué hace Ceocom? El objetivo principal de Ceocom,Qué hace Ceocom?,78,"[-0.016644887626171112, -0.0353408046066761, -...",0.773208


Respuesta obtenida por parte del modelo:
Somos una organización líder en el campo de la Automatización de Procesos, la Inteligencia Artificial y Big Data. Nuestro objetivo es brindar soluciones innovadoras a nuestros clientes para mejorar la eficiencia y la efectividad de sus procesos empresariales. Contamos con un equipo altamente capacitado y experimentado en estas áreas, que trabaja constantemente para ofrecer servicios personalizados y adaptados a las necesidades específicas de cada cliente. Nos enorgullece ser una organización flexible y colaborativa, que trabaja de la mano con nuestros clientes para alcanzar sus objetivos de manera eficaz y eficiente.
